In [1]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from datasets import Dataset, DatasetDict
import torch
from tqdm import tqdm

/home/user/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file = '/tempo/merged_clean.csv'

#convert df into a numpy array
df = pd.read_csv(file)

# convert the relationships into categories
rel_cat = df['Relation'].astype('category')

# label each unique value in the column 'Relation' with a number
labels = rel_cat.cat.codes.to_numpy()
df['Relation'] = labels
conversion_dict = np.array(list(enumerate(rel_cat.cat.categories)))

unique, count = np.unique(labels, return_counts=True)
faulty_labels = unique[count < 50]

# get the indices of falty labels
faulty_indices = [i for i in range(len(labels)) if labels[i] in faulty_labels]
df.drop(faulty_indices, inplace=True)


FileNotFoundError: [Errno 2] No such file or directory: '/tempo/merged_clean.csv'

In [3]:
df.to_csv('/tempo/df.csv', index=False)

NameError: name 'df' is not defined

In [4]:
df = pd.read_csv('processed-files/df.csv')

In [5]:
ds_enc = Dataset.load_from_disk('processed-files/ds_enc')

In [8]:
from transformers import AutoModel, AutoTokenizer
model_ckpt = "sentence-transformers/paraphrase-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained('sbert_model')

In [9]:
num_sent = len(df['Sentence'])
embeddings = np.zeros((num_sent, 384))
for i in tqdm(range(0, num_sent, 100)):
    tokenized = tokenizer(df['Sentence'][i:i+100].to_list(), padding=True, truncation=True, return_tensors="pt")
    embeddings[i:i+100] = model(**tokenized).pooler_output.detach().numpy()


  1%|          | 110/14403 [02:01<4:23:50,  1.11s/it] 


KeyboardInterrupt: 

In [ ]:
np.save('processed-files/fine-tuned_embeddings.npy', embeddings)

: 